In [1]:
# Importing the libraries

In [2]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib as plt 
from sklearn.preprocessing import LabelEncoder
import pickle

In [3]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='br_pR-ZQLsD9VW7QuIEQp-LlNOrLGP31CXwI6vq6HBpu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'carresalevalueprediction-donotdelete-pr-vvbtnese58owrb'
object_key = 'Car Resale Value.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()

,Unnamed: 0,price,yearOfRegistration,monthOfRegistration,powerPS,kilometer,gearbox,notRepairedDamage,model,brand,fuelType,vehicleType
0,0,480,1993,0,0,150000,manual,not-declared,golf,volkswagen,petrol,not-declared
1,1,18300,2011,5,190,125000,manual,Yes,not-declared,audi,diesel,coupe
2,2,9800,2004,8,163,125000,automatic,not-declared,grand,jeep,diesel,suv
3,3,1500,2001,6,75,150000,manual,No,golf,volkswagen,petrol,small car
4,4,3600,2008,7,69,90000,manual,No,fabia,skoda,diesel,small car


In [4]:
labels = ['gearbox', 'notRepairedDamage','model','brand','fuelType','vehicleType']

mapper = {}
for i in labels:
  mapper[i] = LabelEncoder()
  mapper[i].fit(data[i])
  tr = mapper[i].transform(data[i])
  np.save(str('classes'+i+'.npy'), mapper[i].classes_)
  print(i,":",mapper[i])
  data.loc[:,i+"_labels"] = pd.Series(tr, index = data.index)

labeled = data[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration'] + [x+"_labels" for x in labels]]

print(labeled.columns)


gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [5]:
# Splitting Data Into Independent And Dependent Variables

In [6]:
Y = labeled.iloc[:,0:1]
X = labeled.iloc[:,1:11]

In [7]:
X

,yearOfRegistration,powerPS,kilometer,monthOfRegistration,gearbox_labels,notRepairedDamage_labels,model_labels,brand_labels,fuelType_labels,vehicleType_labels
0,1993,0,150000,0,1,2,118,38,7,5
1,2011,190,125000,5,1,1,163,1,1,3
2,2004,163,125000,8,0,2,119,14,1,8
3,2001,75,150000,6,1,0,118,38,7,7
4,2008,69,90000,7,1,0,103,31,1,7
...,...,...,...,...,...,...,...,...,...,...
317374,2005,0,20000,1,2,2,163,33,5,5
317375,2000,101,125000,3,0,0,108,32,7,2
317376,1996,102,150000,3,1,0,226,38,1,0
317377,2002,100,150000,6,1,2,118,38,1,1


In [8]:
Y

,price
0,480
1,18300
2,9800
3,1500
4,3600
...,...
317374,2200
317375,1199
317376,9200
317377,3400


In [9]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=3)

In [10]:
X_train,X_test,Y_train,Y_test

(        yearOfRegistration  powerPS  kilometer  monthOfRegistration  \
 119801                2009      101      40000                   10   
 127219                1998      115     150000                    1   
 13429                 2003      109     150000                    8   
 287795                1999       75     150000                    6   
 258304                2001      140     150000                   12   
 ...                    ...      ...        ...                  ...   
 11261                 2009      116      90000                    3   
 48056                 2005      109     150000                    5   
 77049                 2005      209     150000                    6   
 198296                2007      143     150000                    8   
 71530                 1999      136     150000                    3   
 
         gearbox_labels  notRepairedDamage_labels  model_labels  brand_labels  \
 119801               1                         0    

In [11]:
X_train

,yearOfRegistration,powerPS,kilometer,monthOfRegistration,gearbox_labels,notRepairedDamage_labels,model_labels,brand_labels,fuelType_labels,vehicleType_labels
119801,2009,101,40000,10,1,0,247,36,7,4
127219,1998,115,150000,1,0,1,156,10,5,1
13429,2003,109,150000,8,1,0,28,1,7,7
287795,1999,75,150000,6,1,2,118,38,7,4
258304,2001,140,150000,12,1,0,234,39,7,1
...,...,...,...,...,...,...,...,...,...,...
11261,2009,116,90000,3,1,0,124,24,7,1
48056,2005,109,150000,5,1,0,10,25,1,4
77049,2005,209,150000,6,0,0,246,39,7,8
198296,2007,143,150000,8,1,0,6,2,7,4


In [12]:
Y_train

,price
119801,7499
127219,450
13429,2990
287795,1250
258304,1500
...,...
11261,9390
48056,2900
77049,10500
198296,6995


In [13]:
X_test

,yearOfRegistration,powerPS,kilometer,monthOfRegistration,gearbox_labels,notRepairedDamage_labels,model_labels,brand_labels,fuelType_labels,vehicleType_labels
308948,2006,140,100000,10,1,0,43,24,7,4
259348,2001,179,150000,5,0,0,31,1,1,1
304880,1999,211,150000,7,1,0,167,24,7,1
181015,2001,354,150000,6,0,0,210,20,7,2
202570,2002,104,150000,3,1,2,8,25,7,2
...,...,...,...,...,...,...,...,...,...,...
695,2012,114,40000,8,1,0,226,38,1,0
241345,2007,200,150000,0,0,1,118,38,7,4
3353,2003,113,150000,0,1,2,151,27,4,7
84377,1998,140,150000,1,1,0,40,39,7,4


In [14]:
Y_test

,price
308948,5990
259348,2999
304880,899
181015,15490
202570,1390
...,...
695,28300
241345,7600
3353,2700
84377,850


In [15]:
# Building the model

In [16]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train,Y_train)

/tmp/wsuser/ipykernel_424/3128809939.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train,Y_train)


RandomForestRegressor()

In [17]:
model.score(X_test,Y_test)

0.8467800712920941

In [18]:
y_predict=model.predict(X_test)

In [19]:
from sklearn.metrics import r2_score
print(r2_score(Y_test, y_predict))

0.8467800712920941


In [20]:
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(Y_test, y_predict))

1353.4227172762712


In [21]:
from sklearn.metrics import mean_squared_error

print(mean_squared_error(Y_test, y_predict))

10023883.499039875


In [22]:
from sklearn.metrics import mean_squared_error

root_mean_squared_error = mean_squared_error(Y_test, y_predict, squared=False)
print(root_mean_squared_error)

3166.0517208409397


In [23]:
from sklearn.metrics import r2_score

print(r2_score(Y_test, y_predict))

0.8467800712920941


In [24]:
# SAVING THE MODEL.

In [25]:
import pickle

In [26]:
pickle.dump(model, open("car.pkl", "wb"))

In [27]:
X_train

,yearOfRegistration,powerPS,kilometer,monthOfRegistration,gearbox_labels,notRepairedDamage_labels,model_labels,brand_labels,fuelType_labels,vehicleType_labels
119801,2009,101,40000,10,1,0,247,36,7,4
127219,1998,115,150000,1,0,1,156,10,5,1
13429,2003,109,150000,8,1,0,28,1,7,7
287795,1999,75,150000,6,1,2,118,38,7,4
258304,2001,140,150000,12,1,0,234,39,7,1
...,...,...,...,...,...,...,...,...,...,...
11261,2009,116,90000,3,1,0,124,24,7,1
48056,2005,109,150000,5,1,0,10,25,1,4
77049,2005,209,150000,6,0,0,246,39,7,8
198296,2007,143,150000,8,1,0,6,2,7,4


In [28]:
Y_train


,price
119801,7499
127219,450
13429,2990
287795,1250
258304,1500
...,...
11261,9390
48056,2900
77049,10500
198296,6995


In [29]:
# HOSTING THE MODEL.

In [30]:
import pickle

In [31]:
pickle.dump(model, open("car.pkl", "wb"))

In [32]:
!pip install -U ibm-watson-machine-learning

In [33]:
from ibm_watson_machine_learning import APIClient
import json

In [34]:
# Authenticate and set space.

In [35]:
wml_credentials = {
    "apikey":"qxfbFLITIJs7zvC2zCQmXDBJbJUFCg7XcOjzi8c_VOoA",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [36]:
wml_client = APIClient(wml_credentials)

In [37]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  ----------  ------------------------
ID                                    NAME        CREATED
a749eea7-cd96-45c1-bc5b-9a22b33128fe  Deployment  2022-11-23T04:47:37.716Z
------------------------------------  ----------  ------------------------


In [38]:
SPACE_ID="a749eea7-cd96-45c1-bc5b-9a22b33128fe"

In [39]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [40]:
wml_client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [41]:
# Save and Deploy the model

In [42]:
import sklearn
sklearn.__version__

'1.0.2'

In [43]:
MODEL_NAME = 'Random Forest Regressor'
DEPLOYMENT_NAME = 'Car Resale Value Prediction'
DEMO_MODEL = model

In [44]:
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [45]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [46]:
model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=X_train, 
    training_target=Y_train
)

In [48]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'price',
  'schemas': {'input': [{'fields': [{'name': 'yearOfRegistration',
       'type': 'int64'},
      {'name': 'powerPS', 'type': 'int64'},
      {'name': 'kilometer', 'type': 'int64'},
      {'name': 'monthOfRegistration', 'type': 'int64'},
      {'name': 'gearbox_labels', 'type': 'int64'},
      {'name': 'notRepairedDamage_labels', 'type': 'int64'},
      {'name': 'model_labels', 'type': 'int64'},
      {'name': 'brand_labels', 'type': 'int64'},
      {'name': 'fuelType_labels', 'type': 'int64'},
      {'name': 'vehicleType_labels', 'type': 'int64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-23T05:12:22.409Z',
  'id': 'b8d76fab-aa2d-4151-ade0-dd082a8ce9ec',
  'modified_at': '2022-11-23T05:20:33.175Z',
  'name': 'Random Forest Reg

In [50]:
model_id = wml_client.repository.get_model_id(model_details)

In [51]:
model_id

'b8d76fab-aa2d-4151-ade0-dd082a8ce9ec'

In [52]:
deployment_props = {
   wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
   wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [ ]:
deployment =  wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)



#######################################################################################

Synchronous deployment creation for uid: 'b8d76fab-aa2d-4151-ade0-dd082a8ce9ec' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.........